In [1]:
import pickle
with open ('outfile_Chen', 'rb') as fp:
    news = pickle.load(fp)

In [2]:
import pandas as pd
org = pd.read_csv("temp_Chen.csv")
org.head()

,Unnamed: 0,created_time_taipei,message,jieba_results,like_count
0,0,2020-02-04T20:59:19,新政策上路前夕，整理了選區內的藥局列表，提供給大家，還是要特別提醒～為他人設想不囤貨，把口罩...,新 政策 上路 前夕 整理 了 選區 內 的 藥局 列表 提供 給 大家 還是 要 特別 ...,13021
1,1,2020-02-04T15:50:47,口罩文很多了，現在來聊聊中國滲透WHO事件\n文 / #3Q國會辦公室\n \nWHO秘書長...,口罩 文 很多 了 現在 來 聊聊 中國 滲透 事件 文 / # 國會辦 公室...,27594
2,2,2020-02-04T15:14:15,有人問我國軍支援口罩生產有沒有圖利的問題？\n\n這種邏輯就像是\n救災圖利搬運公司\n打掃...,有人 問我 國軍 支援 口罩 生產有 沒有 圖利 的 問題 這種 邏輯 就 像是 救災 圖...,41113
3,3,2020-02-03T22:37:56,我不知道怎麼回答吃飯為什麼沒有帶口罩的問題，對不起QQ,我 不 知道 怎麼 回答 吃 飯 為 什麼 沒 有 帶 口罩 的 問題 對 不起,17629
4,4,2020-02-03T19:10:01,隨 #武漢肺炎 疫情升溫， 「#你今天買到口罩了嗎？」成為開工後大家最關心的話題之一。\n ...,隨 # 武漢肺炎 疫情 升溫 # 你 今天 買到 口罩 了 嗎 成為 開工 後...,23470


In [3]:
org.index

RangeIndex(start=0, stop=150, step=1)

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
n_topics = 8 # number of topics

In [5]:
cvectorizer = CountVectorizer()
cvz = cvectorizer.fit_transform(news)

# train an LDA model
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=None)
X_topics = lda_model.fit_transform(cvz)

In [6]:
len(X_topics)

150

In [7]:
cvz

<150x12291 sparse matrix of type '<class 'numpy.int64'>'
	with 31080 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.manifold import TSNE
# a t-SNE model
# angle value close to 1 means sacrificing accuracy for speed
# pca initializtion usually leads to better results

In [9]:
tsne_model = TSNE(n_components =2, verbose =1, random_state =0, angle =.99, init='pca')
# 20-D -> 2-D
tsne_lda = tsne_model.fit_transform(X_topics)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 150 samples in 0.000s...
[t-SNE] Computed neighbors for 150 samples in 0.002s...
[t-SNE] Computed conditional probabilities for sample 150 / 150
[t-SNE] Mean sigma: 0.011176
[t-SNE] KL divergence after 250 iterations with early exaggeration: 56.033401
[t-SNE] KL divergence after 850 iterations: 0.092098


In [10]:
import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import save
n_top_words = 10 # number of keywords we show
# 20 colors
colormap = np .array(["#1f77b4","#aec7e8","#ff7f0e","#ffbb78","#2ca02c",
                      "#98df8a","#d62728","#ff9896","#9467bd","#c5b0d5",
                      "#8c564b","#c49c94","#e377c2","#f7b6d2","#7f7f7f",
                      "#c7c7c7","#bcbd22","#dbdb8d","#17becf","#9edae5"])

In [11]:
topicName = []
for i,topic in enumerate(lda_model.components_):
    print(f"TOP #{n_topics} WORDS PER TOPIC #{i}")
    temp = [cvectorizer.get_feature_names()[index] for index in topic.argsort()[-n_top_words:]]
    topicName.append(temp)
    print(temp)

TOP #8 WORDS PER TOPIC #0
['疫情', '口罩', '團隊', '工作', '我們', '車站', '國民黨', '可以', '台灣', '中國']
TOP #8 WORDS PER TOPIC #1
['問題', '疫情', '團隊', '改善', '爭取', '我們', '口罩', '防疫', '台灣', '人員']
TOP #8 WORDS PER TOPIC #2
['大家', '人員', '可能', '相關', '口罩', '防疫', '疫情', '中國', '我們', '台灣']
TOP #8 WORDS PER TOPIC #3
['口罩', '台中市', '組織', '中國', '大家', '烏日', '台灣', '國際', '疫苗', '服務處']
TOP #8 WORDS PER TOPIC #4
['服務', '感染', '服務處', '大家', '口罩', '時間', '防疫', '台灣', '我們', '疫情']
TOP #8 WORDS PER TOPIC #5
['項目', '辦法', '持股', '服務處', '集團', '投資', '沙鹿', '台灣', '我們', '中資']
TOP #8 WORDS PER TOPIC #6
['目前', '大家', '可以', '香港', '防疫', '台電', '需要', '口罩', '我們', '台灣']
TOP #8 WORDS PER TOPIC #7
['企業', '計畫', '表示', '美國', '單位', '疫情', '健康', '政府', '台灣', '疫苗']


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
#然後我們找到每篇文章最可能的主題
_lda_keys = []
color = []
currentTopic = []
top_key = []
for i in range(0, len(X_topics)):
    currentKey = np.argmax(X_topics[i]) #主題分類
    #print(currentKey)
    _lda_keys.append(currentKey)    
    color.append(colormap[currentKey])
    currentTopic.append(topicName[currentKey])

In [13]:
len(tsne_lda[:, 0])

150

In [14]:
len(tsne_lda[:, 1])

150

In [15]:
len(_lda_keys)

150

In [16]:
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

source = ColumnDataSource(dict(
    x = tsne_lda[:, 0],
    y = tsne_lda[:, 1],
    color = color,
    label = currentTopic,
    title = org.index,
    content = org.message
))

Loading BokehJS ...

In [17]:
plot_lda = figure(plot_width=1000, plot_height=600, tools="pan,wheel_zoom,box_zoom,reset,hover",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', source=source, color='color', alpha=0.8, size=10)

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"post": "【num: @title】 【topic: @label】 【content: @content】"}
#plot_lda.legend.location = "top_left"

show(plot_lda)